In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import unidecode #pip install Unidecode

In [2]:
#Função para limpar os dados e obter os nomes dos arquetipos
def get_archetypes(patch):
    data = open(patch,'r',encoding = 'utf8')
    nome_linhas = []
    nome_colunas = []
    index = []
    
    control_flag = 0
    control_count = 0
    
    newMatchup = 0
    newArchetype = ''
    
    for line in data:
        if 'Contra:' in line :
            if control_flag == 0:
                oponent = line[7:].lstrip()
                nome_colunas.append(oponent.rstrip())
                index.append(1)
            else:
                if index[control_count] == 0:
                    oponent = line[7:].lstrip()
                    nome_colunas[control_count] = oponent.rstrip()
                    index[control_count] = 1
            
            
            
        if 'Número de partidas insuficiente' in line or 'Partida espelhada' in line:
            if control_flag == 0:
                nome_colunas.append('missing')
                index.append(0)
        
        
            
        if newMatchup == 1:
            control_count+= 1
            newMatchup = 0
            archetype = line
            
            if archetype != newArchetype and archetype != 'Número de partidas insuficiente\n' :
                if newArchetype != '' and control_flag == 0:
                    control_flag = 1
                
                control_count = 0
                newArchetype = archetype
                nome_linhas.append(newArchetype.rstrip())
                
             
            
        elif line[0] == '~' or line[0].isnumeric() == True:
            newMatchup = 1
    
    return nome_colunas,nome_linhas


In [3]:
#Função para obter os valores númericos
def get_values(patch,mode):
    
    if mode == 0:
        str_values = 'Taxa de vitória:'
        str_end = 16
    else :
        str_values = 'Partidas:'
        str_end = 9
    data = open(patch,'r',encoding = 'utf8')
    values = np.empty(0)
    for line in data :
        if 'Partida espelhada' in line :
            values = np.append(values,'mirror')
        elif str_values in line :
            num = line[str_end:].lstrip().rstrip()
            num = [x for x in num if x.isnumeric()]
            if mode == 0:
                num.insert(2,'.')
            num = ''.join(num)
            values = np.append(values,num)
        elif 'Número de partidas insuficiente' in line :
            values = np.append(values,'noMatches')
    return values,len(values)

In [4]:
def createDatasets(patch):
    # 0 -> taxa de vitoria, 1 -> Número de partidas
    #
    patch = './data.log'
    nome_colunas,nome_linhas = get_archetypes(patch)
    lines,cols = len(nome_linhas),len(nome_colunas)
    
    matriz1 = get_values(patch,1)[0].reshape((49,57))
    matriz0 = get_values(patch,0)[0].reshape((49,57))

    df1 = pd.DataFrame(matriz1,index = nome_linhas, columns= nome_colunas)
    df0 = pd.DataFrame(matriz0,index = nome_linhas, columns= nome_colunas)
    
    #Convert o dataframe em float64 : NaN representa as Mirror Matches
    df1 = df1.apply(pd.to_numeric,args = ('coerce',))
    df0 = df0.apply(pd.to_numeric,args = ('coerce',))
    
    return df0, df1 # % vitorias , N partidas

In [5]:
def metaClasses(df0,df1):
    #Soma os valores das partidas jogadas totais.
    sum_df1 = df1.sum().sort_values(ascending=False)

    #Seleciona os 10 arquétipos mais jogados
    sum_df1_head = sum_df1.head(10)
    
    
    #Obtém os decks com a maior winrate geral com os pesos sendo o número de partidas jogadas contra cada arquétipo
    wr_total = ((df0*df1).transpose().sum()/df1.transpose().sum()).sort_values(ascending = False).head(10)
    
    #Obtém os decks com maior winrate apenas contra os decks mais presentes do meta
    most_played_arq = np.array(sum_df1_head.keys())
    wrdf = df0[most_played_arq].transpose()[most_played_arq]
    sum_arch = df1[wrdf.keys()].transpose()[wrdf.keys()]
    wr_arch = ((wrdf*sum_arch).sum()/sum_arch.sum()).sort_values(ascending = False)
    
    #A partir das análises anteorier adiciona os decks que aparecem em uma mas não em outra.
    classes = np.unique(np.concatenate((np.array(wr_arch.keys()),np.array(wr_total.keys()))))
    
    #Encontra os top counters de cada um dos arquétipos selecionados.
    min_arch= df0.transpose()[classes]
    counters = np.empty(0)
    for i in classes:
        min_wr = np.nanmin(min_arch[i])
        idx = np.where(min_arch[i] == min_wr)
        counters = np.append(counters,df0.keys()[idx])
        
    dici_counters = dict(zip(classes,counters))
    
    #Calcula a winrite dos counters dos principais decks
    tc = df0[classes].transpose()[counters].transpose().drop_duplicates().transpose()
    sum_tc = df1[classes].transpose()[counters].transpose().drop_duplicates().transpose()
    
    wr_tc = ((tc*sum_tc).sum()/sum_tc.sum()).sort_values(ascending = False)
    
    ### Acrescentar as classes que aparecem no top counter mas não nas análises de maior winrate:
    classes = np.unique(np.concatenate((classes,np.array(wr_tc.keys()))))
    return classes

In [6]:
def META(classes,df0,df1):
    #Soma os valores das partidas jogadas totais.
    sum_df1 = df1.sum().sort_values(ascending=False)

    #Seleciona os 10 arquétipos mais jogados
    sum_df1_head = sum_df1.head(10)
    
    #Calcula a winrate total para o as classes agora filtradas.
    wr_total = ((df0*df1).transpose().sum()/df1.transpose().sum())
    wr_total = wr_total[classes].sort_values(ascending = False)
    
    #Calcula o winrate com relação aos decks mais jogados.
    most_played_arq = np.array(sum_df1_head.keys())
    wrdf = df0[most_played_arq].transpose()[classes]

    sum_arch = df1[most_played_arq].transpose()[wrdf.keys()]

    wr_arch = ((wrdf*sum_arch).sum()/sum_arch.sum()).sort_values(ascending = False)
    
    #Cria um dicionário com cada classe e seu respectivo counter à direita.
    min_arch= df0.transpose()[classes]
    counters = np.empty(0)
    for i in classes:
        min_wr = np.nanmin(min_arch[i])
        idx = np.where(min_arch[i] == min_wr)
        counters = np.append(counters,df0.keys()[idx])

    dici_counters = dict(zip(classes,counters))
    
    #Calcula o winrate médio (total/vs decks mais jogados) de cada arquétipo
    mean_wr = ((wr_total+wr_arch)/2).sort_values(ascending = False)

    #Obtém os valores da maior winrate entre os decks counterados caso o arquétipo apareça na lista de counters, 
    #caso não mantém a winrate média.
    keys = list(dici_counters.keys())
    max_counter_wr = dict(mean_wr)
    for key in keys:
        if dici_counters[key] not in classes:
            continue
        counter = dici_counters[key]
        key_wr = mean_wr[key]
        if key_wr > max_counter_wr[counter]:
            max_counter_wr[counter] = key_wr
    #Tira a média aritimética dos valores de counter e a winrate média para definir o META
    max_counter_wr = pd.Series(max_counter_wr)
    META = ((max_counter_wr + mean_wr)/2).sort_values(ascending = False)
    
    return META

In [7]:
#######################################################################################################
# Análise do Trabalho 1 #

### Criação dos Datasets para previsão

In [8]:
def DATASET(patch):
    df0,df1 = createDatasets(patch)
    classes = metaClasses(df0,df1)
    meta = META(classes,df0,df1)
    
    #Obtem as classes que pertencerão ao tier 1
    #classes = [classe for classe,value in  zip(meta.keys(),meta) if value > 50]
    
    #Obter o numero de partidas jogadas:
    #Soma os valores das partidas jogadas totais.
    n_partidas = df1.transpose().sum()
    
    #Winrate Total:
    #Obtém os decks com a maior winrate geral com os pesos sendo o número de partidas jogadas contra cada arquétipo
    wr_total = ((df0*df1).transpose().sum()/df1.transpose().sum())
    
    #Concatena as duas series
    part_winr = pd.concat([wr_total,n_partidas],axis = 1, sort=True)
    part_winr = part_winr.rename(columns ={0:'winrate',1:'partidas'})
    
    #Cria os labels (pertence (1) ou não(0) ao META)
    labels = np.zeros(len(df1.index))-1
    rotulos = list(df1.index)

    serie = pd.Series(labels,rotulos)
    serie[classes] = 1

    #Cria o dataset final
    data = pd.concat([part_winr,serie],axis = 1,sort = False)
    data = data.rename(columns = {0:'meta'})
    return data

In [9]:
DATASET('./data.log')

,winrate,partidas,meta
Big Druid,50.661122,43516.0,-1.0
Big Spell Mage,51.337626,7931.0,-1.0
Bomb Warrior,48.116848,25562.0,-1.0
Bruxo Murloc,45.553109,2454.0,-1.0
Caçador Dragão,54.380436,76701.0,1.0
Control Demon Hunter,44.436723,16964.0,-1.0
Cube Priest,46.027912,17422.0,-1.0
Dragon Druid,50.206510,66853.0,1.0
Druida Malygos,46.394315,8082.0,-1.0
Druida Missão,42.125080,11132.0,-1.0
